In [81]:
import torch
import numpy as np
import torch.nn as nn
from torch.utils.data import DataLoader

import torchvision

from torch.utils.tensorboard import SummaryWriter  # 导入 tensorboard

In [82]:
# 构建 logger 对象，指定 log 文件的保存路径 log_dir='logs'
logger = SummaryWriter(log_dir='logs')

In [83]:
# 此时已经可以通过执行：tensorboard --logdir=path 来访问可视化界面。
# path 填入 logger 对象文件报错路径 例如此处的 './logs'

In [84]:
# 使用 add_scalar 记录标量
for n_iter in range(100):
    logger.add_scalar(tag='Loss/train', scalar_value=np.random.random(), global_step=n_iter)
    logger.add_scalar(tag='Loss/test', scalar_value=np.random.random(), global_step=n_iter)

In [85]:
# %load_ext tensorboard
# %tensorboard --logdir='logs'

In [86]:
# !kill 75314  # 关闭 tensorboard

In [87]:
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))  # MNIST 是灰度图，单通道
])

# './data/' 指定下载的路径，download=True 表示下载，train=False 表示下载测试集，transform 指定变换
dataset = datasets.MNIST('./data/', download=True, train=False, transform=transform)
dataloader = DataLoader(dataset, shuffle=True, batch_size=16)

In [88]:
images, labels = next(iter(dataloader))
grid = torchvision.utils.make_grid(images)

In [89]:
images.shape

In [90]:
# 使用 add_image 显示图像
logger.add_image('images', grid, 0)

In [91]:
class myModel(nn.Module):
    def __init__(self, input_size=28, hidden_size=500, num_classes=10):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out


model = myModel()
logger.add_graph(model, images)  # 代入具体的数值计算 torch.Size([16, 1, 28, 28])

In [92]:
logger.add_histogram('normal', np.random.normal(0, 5, 1000), global_step=1)
logger.add_histogram('normal', np.random.normal(1, 2, 1000), global_step=10)

In [94]:
# 使用 add_embedding 进行 embedding 可视化
dataset = datasets.MNIST('./data/', download=True, train=False)  # PIL Image 对象可以可视化
images = dataset.data[:100].float()  # 提取前 100 张图像并转为浮点数类型
label = dataset.targets[:100]  # 提取前 100 张图像的标签
features = images.view(100, 28 * 28)  # 将图像展平为 100 x 784 的矩阵
logger.add_embedding(features, metadata=label, label_img=images.unsqueeze(1))  # 将嵌入数据记录到 TensorBoard